<a href="https://colab.research.google.com/github/AaryanBairagi/GeekForGeeks-21Days-21Projects-Challenge/blob/main/Day_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install langchain
! pip install langchain-community
! pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.9 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:


# **TESTING- Simple CHATBOT**

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage , SystemMessage
from google.colab import userdata

chat_model = ChatGoogleGenerativeAI(model='gemini-2.5-flash',
                                    temperature=0,
                                    google_api_key=userdata.get('smart_IDP'))

messages = [
    SystemMessage("You are an assistant knowledgable about Healthcare. Only answer Healthcare-related questions."),
    HumanMessage(content="What is Pulmonary Artery?")
    ]

result = chat_model.invoke(messages)
print(result.content)


The **Pulmonary Artery** is a major blood vessel that originates from the right ventricle of the heart and carries deoxygenated blood to the lungs.

Here's a breakdown of its key characteristics and function:

1.  **Origin:** It emerges from the right ventricle of the heart.
2.  **Blood Type:** Unlike most other arteries in the body (which carry oxygenated blood), the pulmonary artery carries **deoxygenated blood** (blood that has already delivered its oxygen to the body's tissues and is rich in carbon dioxide).
3.  **Destination:** It branches into the right and left pulmonary arteries, which then further divide into smaller arteries and capillaries within the lungs.
4.  **Function:** Its primary role is to transport this deoxygenated blood to the lungs, where it can release carbon dioxide and pick up fresh oxygen through the process of gas exchange in the alveoli.
5.  **Unique Feature:** It is the only artery in the adult body that carries deoxygenated blood. Once the blood is oxygen

# **TESTING-Using Chains and PromptTemplates**

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from google.colab import userdata
from langchain_core.output_parsers import StrOutputParser

chat_model = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=0,
    google_api_key=userdata.get('smart_IDP')
)

instruction_str = """Your job is to use patient reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed as possible, but don't make up any information that's not from the context.
If you don't know an answer, say you don't know.

Context: {context}

Question: {question}
"""

chat_template = ChatPromptTemplate.from_template(instruction_str)

context = "Discharge process was seamless!"
question = "Did anyone have a positive experience?"

chain = chat_template | chat_model | StrOutputParser()

result = chain.invoke({
    "context":context,
    "question":question
})

print(result)

Yes, someone had a positive experience. Their discharge process was seamless, indicating a smooth and problem-free experience.


# **Using PromptTemplates & MessageTemplates**

In [4]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (ChatPromptTemplate , SystemMessagePromptTemplate , HumanMessagePromptTemplate , PromptTemplate)

chat_model = ChatGoogleGenerativeAI(model='gemini-2.5-flash',
                                    temperature=0,
                                    google_api_key=userdata.get('smart_IDP'))

instruction_str = """Your job is to use patient reviews to answer questions about their experience at a hospital.
Use the following context to answer questions.
Be as detailed as possible, but don't make up any information that's not from the context.
If you don't know an answer, say you don't know.

Context: {context}
"""

chat_system_prompt_template = SystemMessagePromptTemplate(
    prompt = PromptTemplate(input_variables=["context"] , template=instruction_str)
)


chat_human_prompt_template = HumanMessagePromptTemplate(
    prompt = PromptTemplate(input_variables=["question"] , template="{question}")
)

messages=[chat_system_prompt_template,chat_human_prompt_template]

chat_prompt_template = ChatPromptTemplate(
    input_variables=["context","question"],
    messages=messages
)

context = "I had a great stay!"
question = "Did anyone have a positive experience?"

chain = chat_prompt_template | chat_model | StrOutputParser()

result = chain.invoke({
    "context":context,
    "question":question
})

print(result)

Yes, one patient stated, "I had a great stay!"


# **Adding RAG**

In [5]:
! pip install langchain-chroma
! pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 7.7 MB/s eta 

In [6]:
# import time
# from google.colab import userdata
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain_chroma import Chroma
# from langchain.document_loaders import CSVLoader

# review_csv_path = "/content/reviews.csv"

# review_chroma_path = "chroma_data"

# loader = CSVLoader(
#     file_path = review_csv_path,
#     source_column = 'review'
# )

# reviews = loader.load()

# embedding_function = GoogleGenerativeAIEmbeddings(
#     model='models/gemini-embedding-001',
#     google_api_key = userdata.get("smart_IDP")
# )

# batch_size = 20

# num_batches = (len(reviews)-1) // batch_size+1
# reviews_verctor_db = None

# for i in range(0,len(reviews),batch_size):
#   batch_docs = reviews[i:i+batch_size]
#   current_batch = i // batch_size + 1

#   print(f"Processing batch {current_batch}/{num_batches}...")
#   if i==0:
#     reviews_verctor_db = Chroma.from_documents(
#         documents=batch_docs,
#         embedding=embedding_function,
#         persist_directory=review_chroma_path
#     )

#   else:
#     reviews_verctor_db.add_documents(documents=batch_docs)

#   print(f"Batch {current_batch} processed. Waiting for 30 seconds...")
#   time.sleep(30)

# print("Vector database created successfully and saved to the specified directory.")


# **PUTTING IT ALL TOGETHER**

In [1]:
from langchain_core.prompts import( SystemMessagePromptTemplate , HumanMessagePromptTemplate , ChatPromptTemplate , PromptTemplate)
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain.document_loaders import CSVLoader
import time
from langchain.embeddings import HuggingFaceEmbeddings


instruction_str = """Your job is to use patient reviews to answer questions about their experience at a hospital.
Use the following context to answer questions.
Be as detailed as possible, but don't make up any information that's not from the context.
If you don't know an answer, say you don't know.

{context}
"""

reviews_system_prompt_template = SystemMessagePromptTemplate(
    prompt = PromptTemplate(input_variables=["context"],
    template = instruction_str)
)


reviews_human_prompt_template = HumanMessagePromptTemplate(
    prompt = PromptTemplate(input_variables=["question"],
    template = "{question}")
)


messages = [reviews_system_prompt_template , reviews_human_prompt_template]


reviews_chat_prompt_template = ChatPromptTemplate(
    input_variables=["context","question"],
    messages = messages
)


print(f"Creating Chat Model...")

reviews_chat_model = ChatGoogleGenerativeAI(
    model = 'gemini-2.5-flash',
    temperature = 0,
    google_api_key = userdata.get('RAGBOT_API_KEY')
)


print(f"Chat Model Created.")


reviews_cs_path = "/content/reviews.csv"


loader = CSVLoader(
    file_path = reviews_cs_path,
    source_column = 'review'
)


reviews = loader.load()


reviews_chroma_path = "chroma_data"


# embedding_function = GoogleGenerativeAIEmbeddings(
#     model="models/gemini-embedding-001",
#     google_api_key=userdata.get("RAGBOT_API_KEY")
# )


embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



batch_size = 20


num_of_batches = (len(reviews) - 1 ) // batch_size + 1


for i in range(0,len(reviews),batch_size):
  batch_docs = reviews[i:i+batch_size]
  current_batch = i // batch_size+1

  print(f"Processing batch {current_batch}/{num_of_batches}...")

  if i==0:
    reviews_vector_db = Chroma.from_documents(
        documents = batch_docs,
        embedding = embedding_function,
        persist_directory = reviews_chroma_path
    )
  else:
    reviews_vector_db.add_documents(documents=batch_docs)

  print(f"Batch {current_batch} processed. Waiting for 30 seconds...")
  time.sleep(30)

print("Vector database created successfully and saved to the specified directory")



Creating Chat Model...
Chat Model Created.


/tmp/ipython-input-2776224186.py:73: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public m

Processing batch 1/51...
Batch 1 processed. Waiting for 30 seconds...
Processing batch 2/51...
Batch 2 processed. Waiting for 30 seconds...
Processing batch 3/51...
Batch 3 processed. Waiting for 30 seconds...
Processing batch 4/51...
Batch 4 processed. Waiting for 30 seconds...
Processing batch 5/51...
Batch 5 processed. Waiting for 30 seconds...
Processing batch 6/51...
Batch 6 processed. Waiting for 30 seconds...
Processing batch 7/51...
Batch 7 processed. Waiting for 30 seconds...
Processing batch 8/51...
Batch 8 processed. Waiting for 30 seconds...
Processing batch 9/51...
Batch 9 processed. Waiting for 30 seconds...
Processing batch 10/51...
Batch 10 processed. Waiting for 30 seconds...
Processing batch 11/51...
Batch 11 processed. Waiting for 30 seconds...
Processing batch 12/51...
Batch 12 processed. Waiting for 30 seconds...
Processing batch 13/51...
Batch 13 processed. Waiting for 30 seconds...
Processing batch 14/51...
Batch 14 processed. Waiting for 30 seconds...
Processing

In [2]:
from langchain.schema.runnable import RunnablePassthrough  # Allows passing inputs through unchanged in a pipeline
from langchain_core.output_parsers import StrOutputParser

reviews_retriever = reviews_vector_db.as_retriever(k=10)

review_chain = (
    {"context":reviews_retriever , "question":RunnablePassthrough()} | reviews_chat_prompt_template | reviews_chat_model | StrOutputParser()
)

In [3]:
question = """Has anyone complained about communication with the hospital staff?"""
review_chain.invoke(question)

'No, based on the provided reviews, no one has complained about communication with the hospital staff. The complaints in these reviews are related to the noise level in the ward or shared room.'

In [4]:
! pip install gradio

In [5]:
def respond_to_user_question(question:str , history:list) -> str:
  """
  Respond the user's question using the review_chain
  """
  return review_chain.invoke(question)

In [6]:
respond_to_user_question("Has anyone complaied about communication with the hospital staff?",[])

'Yes, Dennis Fitzgerald complained about communication with the hospital staff.\n\nSpecifically, Dennis Fitzgerald stated that "The hospital staff lacked proper communication among themselves, leading to confusion about my treatment plan." They emphasized that "Clear and cohesive communication is essential for patient care and overall satisfaction."'

In [7]:
import gradio as gr
interface = gr.ChatInterface(fn=respond_to_user_question, title="Review Helper Bot")

interface.launch(debug=True)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:348: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6ab45fa9e39d625ff3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6ab45fa9e39d625ff3.gradio.live
